# Task 1 a) Process Quadrat in MP4 Video
This notebook is configured to read a video specified by `file`, then extract the lines that compose the quadrat and estimate the appropriate corner points for cropping its 
contents. The output is a lower resolution MP4 video that is annotated with the found lines 
and corner points.

In [ ]:
import os
import os.path as osp
import sys

# Check if notebook is running in Colab or local workstation
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Search for all video files on Google Drive...
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = r'/content/drive/My Drive/Data'
    
    # cd into current directory so local imports work
    %cd '/content/drive/My Drive/cciw-zebra-mussel/quadrat-extraction/'
    
    # clone repo, install packages
else:
    DATA_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/Data')
    SAVE_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/dataset_raw/quadrat-extraction/videos')

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
#from utils import draw_lines, draw_all_lines
from utils import crop_still_image
from utils import crop_still_image_no_rotate
from utils import compute_pairwise_distances

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
#%load_ext autoreload
#%autoreload 2

figsize = (8, 6)
save_figures = False

In [ ]:
file = 'GLNI_12-1_2015-07-09_video-1.mp4'
#file = 'GLNI_456-3_2015-07-17_video-1.mp4'
#file = 'GLNI_1347-2_2013-09-13_video-1.mp4'

#file = 'GLNI_1347-1_2015-07-15_video-1.mp4'
#file = 'GLNI_1347-3_2015-07-15_video-1.mp4'
#file = 'GLNI_1347-3_2015-07-02_video-1.mp4'
#file = 'GLNI_1348-2_2015-07-16_video-1.mp4'

#file = 'GLNI_1347-3_2013-07-11_video-1.mp4'

In [ ]:
all_videos = glob(os.path.join(DATA_PATH, 'Videos_and_stills/GLNI/*/*/*/Videos/Quad*/*.mp4'))
videotable_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
video_df = pd.read_csv(videotable_path, index_col=0)

vpath = video_df.iloc[video_df[video_df['Name'] == file].index]['Quadrat Video Path']
tokens = video_df[video_df['Name'] == file]['Quadrat Video Path'].values[0].split('\\')

video_path = DATA_PATH + '/Videos_and_stills/GLNI'
for tok in tokens[4:-1]:
    video_path += '/' + tok
video_path = os.path.join(video_path, file)

print('Found %d videos' % len(all_videos))
print('Loading video: ', video_path)

In [ ]:
output_folder = os.path.join(SAVE_PATH, file.split('.')[0])
if not osp.exists(output_folder):
    os.makedirs(output_folder)
    print('Made output folder ', output_folder)
else:
    print('Output folder %s already exists' % output_folder)

In [ ]:
"""These are meta-parameters of the Probabilistic Hough Line Transform, 
note there are additional params in cell 6 which we set according to 
the input resolution.

@param rho Distance resolution of the accumulator (pixels)"""
rho = 1

"""
@param theta Angle resolution of the accumulator (radians)

Suggest to use a value no less than np.pi/90 else too many 
spurious lines will be found. theta=np.pi/45 means lines 
must differ by 4 degrees to be considered distinct."""
#theta = np.pi / 45

In [ ]:
cap = cv2.VideoCapture(video_path)
sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
      int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('Raw input resolution', sz)

In [ ]:
DRAW = True
SAVE_STILLS = True # save still images cropped from Quadrat interior when True
SAVE_VIDEO = False # save video with quadrat annotated on top when True.

# indices of data in x and y position respectively
X, Y = 0, 1

In [ ]:
# read first frame to adjust resolution of output stream
ret, im = cap.read() 

# set additional meta-parameters according to input res
if sz[0] == 1440:
    """x_trim and y_trim are used to remove black padding 
    which triggers spurious edges"""
    x_trim, y_trim = 1, 145
    im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    crop_frame_border = True
    
    """@param canny_thresh# hysteresis values for Canny edge 
    detector, input to HoughLines"""
    #canny_thresh1, canny_thresh2 = 60, 300 # v0
    canny_thresh1, canny_thresh2 = 10, 45
    
    """@param threshold Accumulator threshold, return 
    lines with more than threshold of votes. (intersection points)"""
    threshold = 125
    
    """@param minLineLength Minimum line length. 
    Line segments shorter than that are rejected. (pixels)"""
    mLL = 400
    
    """@param maxLineGap Maximum allowed gap between points 
    on the same line to link them. (pixels)"""
    mLG = 150
else:
    # params as described above
    #canny_thresh1, canny_thresh2 = 30, 300 # v0
    canny_thresh1, canny_thresh2 = 40, 100
    threshold = 125
    #mLG, mLL = 250, 600
    mLG, mLL = 150, 500
    crop_frame_border = False

"""this method may downsample, so set the video writer 
resolution to the processed image resolution"""
#img, edges, c2 = draw_all_lines(im, rho=rho, theta=theta, mll=mLL, mlg=mLG, threshold=threshold, ds=1)    
img, edges, crop = crop_still_image_no_rotate(
        im, mll=mLL, mlg=mLG, threshold=threshold, canny_1=canny_thresh1, canny_2=canny_thresh2, do_draw=DRAW)
sz = (img.shape[1], img.shape[0])
print(sz)

In [ ]:
#for i in range(100):
#    ret, im = cap.read() 
plt.imshow(img)

In [ ]:
if cap.isOpened():
    if SAVE_VIDEO:
        fps = 20
        vout = cv2.VideoWriter()
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        vout.open(file.split('.')[0] + '-quadrat-demo.mp4', fourcc, fps, sz, True)
        print('Saving as video ', file.split('.')[0] + '-quadrat-demo.mp4')
    print('Opened stream for writing, output resolution is', sz)
else:
    print('cap is not open')

In [ ]:
skipFrames = 10
currentFrame = 0

"""it can take 30s-1min to process entire video, 
can optionally process a small number of frames"""

# to process whole video    
while(True):
    
    # Capture frame-by-frame
    
    for i in range(skipFrames):
        ret, im = cap.read()
    
    if not ret: break
    
    if crop_frame_border:
        im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    
    # Do processing
    '''
    img, _ = draw_lines(im, rho=rho, theta=theta, mll=mLL, 
                        mlg=mLG, threshold=threshold, ds=1, 
                        canny_1=canny_thresh1, canny_2=canny_thresh2)
    img, _, _ = draw_all_lines(im, rho=rho, theta=theta, mll=mLL, 
                           mlg=mLG, threshold=threshold, ds=1, 
                           canny_1=canny_thresh1, canny_2=canny_thresh2)
    '''
    if SAVE_VIDEO:
        img, _, _ = crop_still_image_no_rotate(
        im, mll=mLL, mlg=mLG, threshold=threshold, canny_1=canny_thresh1, canny_2=canny_thresh2, do_draw=DRAW)
    
    if SAVE_STILLS:
        img, _, crop = crop_still_image(
            im, mll=mLL, mlg=mLG, threshold=threshold, canny_1=canny_thresh1, canny_2=canny_thresh2, do_draw=DRAW)
    
    # save still image in jpeg format
    if SAVE_STILLS:
        try:
            x_start = crop[:, X].min()
            x_end = crop[:, X].max()
            y_start = crop[:, Y].min()
            y_end = crop[:, Y].max()

            if (compute_pairwise_distances(crop)[:, 2] < mLL).sum():
                print('Cannot crop: found corners do not form a square')
            else:
                try:
                    out_file = osp.join(
                        output_folder, file.split('.')[0] + '_crop_frame-%d.jpg' % currentFrame)
                    cv2.imwrite(out_file, img[y_start:y_end, x_start:x_end, :])
                except:
                    print('Cannot write ', out_file)
        except:
            pass
            print('Cannot crop: insufficient number of corner points found')
    
    """For annotating video
    @param org Bottom-left corner of the text string (default=50).
    @param org Bottom-left corner of the text string in the image (default=50).
    @param fontFace Font type, see #HersheyFonts (default=cv2.FONT_HERSHEY_PLAIN).
    @param fontScale Font scale factor that is multiplied 
                     by the font-specific base size (default=2).
    @param color Text color (default=(R=0, G=255, B=0)).
    @param thickness Thickness of the lines used to draw a text (default=1).
    @param lineType Line type. See #LineTypes (default=cv2.LINE_AA)."""
    cv2.putText(                 # x, y
        img, str(currentFrame), (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 1, cv2.LINE_AA)
    if SAVE_VIDEO:
        vout.write(img)

    # increment frame counter
    currentFrame += skipFrames
    
    if currentFrame > 600:
        break

"""When everything done, release the 
capture to flush the video stream. 

Before re-running this cell, first do 
cells 5, 6, and 7 to re-open cap and 
vout, otherwise ret=False and this 
cell does nothing."""
cap.release()
if SAVE_VIDEO:
    vout.release()

You should see new mp4 files when listing current dir. You may not be able to view them directly in Google Drive,
but these can be downloaded to your local machine.